
##Covid effects on...


In [2]:
import pandas as pd
from pandas import json_normalize
import json
import matplotlib.pyplot as plt
import folium
import requests
import logging
from datetime import date
import time
import asyncio
import aiohttp
import nest_asyncio
import plotly.graph_objects as go
import numpy as np


In [7]:
covid_dataframe = pd.read_csv('covid_tracking_13_may.csv')
covid_dataframe['date'] = covid_dataframe['date'].apply(lambda x: pd.to_datetime(str(x), format='%Y-%m-%d')) 
covid_dataframe.head(2)

,Unnamed: 0,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,...,hospitalized,total,totalTestResults,posNeg,fips,deathIncrease,hospitalizedIncrease,negativeIncrease,positiveIncrease,totalTestResultsIncrease
0,0,2020-05-13,AK,383.0,30266.0,NaN,12.0,NaN,NaN,NaN,...,NaN,30649,30649,30649,2,0.0,0.0,688.0,0.0,688.0
1,1,2020-05-13,AL,10617.0,125755.0,NaN,NaN,1317.0,NaN,477.0,...,1317.0,136372,136372,136372,1,20.0,30.0,2847.0,307.0,3154.0


In [4]:
industriesDF = pd.read_csv('merged_industries.csv')
industriesDF.head(2)

In [4]:

groupedDF = covid_dataframe.groupby(['date'])
positiveseries = groupedDF['positive'].sum()
positiveseries.plot(figsize=(12,8), linewidth=5, legend=False)
plt.grid(color="gray", linestyle='-', linewidth=1)
plt.title("Total Positive US Covid19 Cases Over Time", fontsize=20)
plt.ylabel("Number of Cases", fontsize=18)
plt.xlabel("Month", fontsize=18)
plt.show()

,Unnamed: 0,Symbol,Security,GICS Sector,GICS Sub Industry,Headquarters Location,Date first added,GICS Codes/ Sub-industries,NAICS
0,0,MMM,3M Company,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,Industrial Conglomerates,Manufacturing
1,1,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,Health Care Equipment,Health care and social assistance


In [5]:
#This creates the final stockdata dataframe
stockDF = pd.read_csv('alphavantage_stock_prices_current_csv.csv')
stockDF.head(2)

,symbol,2020-05-13.4. close,2020-05-12.4. close,2020-05-11.4. close,2020-05-08.4. close,2020-05-07.4. close,2020-05-06.4. close,2020-05-05.4. close,2020-05-04.4. close,2020-05-01.4. close,...,2019-12-31.4. close,2019-12-30.4. close,2019-12-27.4. close,2019-12-26.4. close,2019-12-24.4. close,2019-12-23.4. close,2019-12-20.4. close,2019-12-19.4. close,2019-12-18.4. close,2019-12-17.4. close
0,CBRE,34.84,37.20,40.82,41.600,40.89,38.15,39.59,39.390,39.70,...,61.29,61.04,60.95,61.28,60.59,60.11,60.64,59.82,NaN,NaN
1,CMCSA,34.97,35.35,36.16,36.535,35.58,35.14,36.05,36.215,36.98,...,44.97,45.18,45.10,44.97,44.53,44.10,44.09,43.64,NaN,NaN


In [5]:
master_industryDF = pd.read_csv('workers_by_indust.csv')
master_industryDF

,Unnamed: 0,NAME,Code,STATE,LineCode,Description,2018 Workers In Industry,2018 Working Pop,2018 Overall Population,2019 Overall Population,2018 Percentage of Working Pop
0,0,Alabama,AL,1,70,Farm employment,42220.0,2691517,4887871,4903185,1.568632
1,1,Alabama,AL,1,100,"Forestry, fishing, and related activities",15895.0,2691517,4887871,4903185,0.590559
2,2,Alabama,AL,1,200,"Mining, quarrying, and oil and gas extra...",10560.0,2691517,4887871,4903185,0.392344
3,3,Alabama,AL,1,300,Utilities,14176.0,2691517,4887871,4903185,0.526692
4,4,Alabama,AL,1,400,Construction,148326.0,2691517,4887871,4903185,5.510870
...,...,...,...,...,...,...,...,...,...,...,...
1062,1066,Wyoming,WY,56,1600,Health care and social assistance,30617.0,405010,577737,578759,7.559566
1063,1067,Wyoming,WY,56,1700,"Arts, entertainment, and recreation",8255.0,405010,577737,578759,2.038221
1064,1068,Wyoming,WY,56,1800,Accommodation and food services,35917.0,405010,577737,578759,8.868176
1065,1069,Wyoming,WY,56,1900,Other services (except government and go...,17920.0,405010,577737,578759,4.424582


In [6]:
master_unemp_DF = pd.read_csv('bls_unemployment_rate_2010-2020.csv')
master_unemp_DF

,Unnamed: 0,Unnamed: 1,value,value.1,value.2,value.3,value.4,value.5,value.6,value.7,value.8,value.9,value.10
0,year,NaN,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0
1,state,month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AL,1,12.3,11.0,8.3,8.2,7.6,6.4,6.3,6.0,4.3,4.2,3.2
3,AL,2,12.1,10.5,8.4,7.9,7.8,6.3,6.2,5.6,4.4,3.8,2.9
4,AL,3,11.5,9.9,7.9,7.1,7.3,5.9,5.9,4.9,4.0,3.4,3.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
585,WY,8,8.6,7.9,7.6,6.6,6.6,6.7,6.0,5.3,5.0,4.8,NaN
586,WY,9,8.1,7.5,7.0,6.0,5.9,5.9,5.4,4.6,4.5,4.3,NaN
587,WY,10,8.0,7.2,7.0,6.2,5.8,5.7,5.3,4.6,4.5,4.6,NaN
588,WY,11,8.3,7.1,7.0,6.3,6.0,5.9,5.1,4.9,4.5,4.8,NaN
